In [1]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
import pandas as pd
import sys

sys.path.append(os.getcwd())
import tsa_utils

from shutil import copytree
import numpy as np
import matplotlib as mpl

import sklearn
from sklearn import metrics

import pip

from sklearn.model_selection import train_test_split
pip.main(['install', '--upgrade', 'dask'])
pip.main(['install', 'objgraph'])

import objgraph

The directory '/Users/jasminefeldmann/Library/Caches/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/Users/jasminefeldmann/Library/Caches/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


Requirement already up-to-date: dask in /Users/jasminefeldmann/anaconda/lib/python3.6/site-packages


The directory '/Users/jasminefeldmann/Library/Caches/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/Users/jasminefeldmann/Library/Caches/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [2]:
# copytree("/models/multi-class8/multi_class", "/output/multi_class")

In [14]:
#Model parameters
BATCH_SIZE=10
FILTER_COUNT=30
KERNEL_SIZE1=(1,5,5)
DEPTHSTRIDE=1
XSTRIDE=1
YSTRIDE=1
POOLSIZE1=(1,3,3)
POOLSIZE2=(1,5,5)
POOL_STRIDES1=(1,2,2)
POOL_STRIDES=(1,1,1)
STEPS=10000
XSIZE=256
#XSIZE=512
#XSIZE=270
#YSIZE=340
YSIZE=330
#YSIZE=660
LEARNING_RATE=0.0001
EPSILON=0
IMAGE_DEPTH=16
CHANNELS=1
FLAT_POOL_SIZE=198720
REGULARIZER_VALUE=0.01

DATA_PATH="/a3d_volume/"
CHECKPOINT_PATH="/output/multi_class"
MODEL_ID="multi_class3d10"

tf.logging.set_verbosity(tf.logging.INFO)
os.makedirs("/output/tmp/")

FileExistsError: [Errno 17] File exists: '/output/tmp/'

In [12]:
class ZoneModel():

    def __init__(self, model_id, ids, x_slice, y_slice, data_path, labels, checkpoint_path=".", localize=False,scaler_ids=None, iteratorClass=tsa_utils.InputImagesIterator, test_ids=None, test_labels=None, randomize=False, seed=191):
        self.model_id = model_id
        self.ids = ids
        self.x_slice = x_slice
        self.y_slice = y_slice
        self.data_path = data_path
        self.checkpoint_path = checkpoint_path
        self.labels = labels
        self.localize = localize
        self.scaler_ids=scaler_ids
        self.iteratorClass = iteratorClass
        self.test_ids = test_ids
        self.test_labels = test_labels
        self.randomize=randomize
        self.seed=seed
        

    def build_model(self, data, labels, mode):
        regularizer = tf.contrib.layers.l2_regularizer(REGULARIZER_VALUE)
        print(regularizer)
        if mode != tf.contrib.learn.ModeKeys.TRAIN:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=10
            
        training = (mode==tf.contrib.learn.ModeKeys.TRAIN)
        print(training)
        print(XSIZE, YSIZE)
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
#         normalizer = tf.contrib.layers.batch_norm(
#             data,
#             data_format='NHWC',  # Matching the "cnn" tensor which has shape (?, 9, 120, 160, 96).
#             center=True,
#             scale=True,
#             is_training=training,
#             scope='bn')

        dropout = tf.layers.dropout(data, rate=0.3, training=training)
        conv1 = tf.layers.conv3d(inputs=dropout, filters=FILTER_COUNT, kernel_size=KERNEL_SIZE1, padding="same", 
                strides=(DEPTHSTRIDE,XSTRIDE,YSTRIDE), name="conv1", trainable=not self.localize, kernel_regularizer=regularizer, bias_regularizer=regularizer)
        print(conv1)
#         pool0 = tf.layers.max_pooling3d(inputs=conv1, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool0")
#         print(pool0)
#         conv2 = tf.layers.conv3d(inputs=conv1, filters=FILTER_COUNT, kernel_size=(1,3,3), padding="same",
#                 strides=(1, 2, 2), name="conv2", activation=tf.nn.relu, trainable=not self.localize, kernel_regularizer=regularizer)
#         print(conv2)
#         pool1 = tf.layers.max_pooling3d(inputs=conv2, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool1")
#         print(pool1)
#         conv3 = tf.layers.conv3d(inputs=pool1, filters=FILTER_COUNT, kernel_size=(1,3,3), padding="same", 
#                 strides=(DEPTHSTRIDE,2,2), name="conv3", trainable=not self.localize, kernel_regularizer=regularizer)
#         print(conv3)
        pool2 = tf.layers.max_pooling3d(inputs=conv1, pool_size=(1,4,4), strides=(1,2,2), name="pool2")
        print(pool2)
        conv4 = tf.layers.conv3d(inputs=pool2, filters=FILTER_COUNT, kernel_size=(1,3,3), padding="same", 
                strides=(1, 1, 1), name="conv4", activation=tf.nn.relu, trainable=not self.localize, kernel_regularizer=regularizer, bias_regularizer=regularizer)
        print(conv4)
        conv5 = tf.layers.conv3d(inputs=conv4, filters=FILTER_COUNT, kernel_size=(1,3,3), padding="same", 
            strides=(1, 1, 1), name="conv5", activation=tf.nn.relu, trainable=not self.localize, kernel_regularizer=regularizer, bias_regularizer=regularizer)
        print(conv4)
        pool3 = tf.layers.max_pooling3d(inputs=conv5, pool_size=(1,7,7), strides=(1,7,7), name="pool3")
        print(pool3)
        flat_pool = tf.reshape(pool3, [BATCH_SIZE, FLAT_POOL_SIZE])
        flat_pool=tf.identity(flat_pool, name="flat_pool")
        print(flat_pool)
#         dense = tf.layers.dense(inputs=flat_pool, units=20, trainable=True, activation=tf.nn.relu, kernel_regularizer=regularizer, bias_regularizer=regularizer)
#         print(dense)
        dropout2 = tf.layers.dropout(flat_pool, rate=0.3, training=training)
        print(dropout2)
        logits = tf.layers.dense(inputs=dropout2, units=17, trainable=True, kernel_regularizer=regularizer, bias_regularizer=regularizer)
        logits = tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,17])
        reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        reg_losses = tf.identity(reg_losses, "reg_losses")
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.sigmoid(
              logits, name="prob_tensor")}
        print(mode)
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        #flat_labels = tf.reshape(labels, [BATCH_SIZE, 17])
        labels=tf.identity(labels, name="labels")
        labels = tf.cast(labels, tf.float32)
        #class_weights=tf.reduce_sum(tf.multiply(flat_labels, tf.constant(WEIGHTS, dtype=tf.int64)), axis=1)
        #print(class_weights)
        #loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits)) + sum(
                    tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
        else:
            print("RUNNGIN IN MODE " + str(mode))
            loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits))
#         update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#         with tf.control_dependencies(update_ops):
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=LEARNING_RATE,
            optimizer="SGD")

        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

    def train_model(self, tensors_to_log, reuse=False):
        if reuse:
            tsa_classifier = self.model
        else:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                     model_dir=self.checkpoint_path + "/" + self.model_id)
            
#         validation_metrics = {
#         "accuracy":
#             tf.contrib.learn.MetricSpec(
#                 metric_fn=tf.contrib.metrics.streaming_accuracy,
#                 prediction_key=tf.cast(tf.reshape(tf.contrib.learn.PredictionKey.CLASSES,[1,1]), tf.bool).eval()),
#         "precision":
#             tf.contrib.learn.MetricSpec(
#                 metric_fn=tf.contrib.metrics.streaming_precision,
#                 prediction_key=tf.cast(tf.reshape(tf.contrib.learn.PredictionKey.CLASSES, [1,1]), tf.bool).eval()),
#         "recall":
#             tf.contrib.learn.MetricSpec(
#                 metric_fn=tf.contrib.metrics.streaming_recall,
#                 prediction_key=tf.cast(tf.reshape(tf.contrib.learn.PredictionKey.CLASSES, [1,1]), tf.bool).eval())
#         }
        
#         validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
#             self.iteratorClass(self.test_ids, self.data_path, repeating=False, pool_size=2),
#             tsa_utils.InputLabelsIterator(self.test_ids, self.test_labels),
#             every_n_steps=100, early_stopping_rounds=300, metrics=validation_metrics)
        
        logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
        #f1_hook = calculate_f1(self.model)
        tsa_classifier.fit(
            #x=tsa_utils.ThresholdedInputImagesIterator(self.ids, self.data_path, 10000, self.y_slice, self.x_slice, pool_size=2), 
            x=self.iteratorClass(self.ids, self.data_path, repeating=True, pool_size=2, file_format=".a3daps", randomize=self.randomize, seed=self.seed),
            y=tsa_utils.InputLabelsIterator(self.ids, self.labels, randomize=self.randomize, seed=self.seed), 
            max_steps=STEPS, 
            batch_size=BATCH_SIZE, 
            #monitors=[logging_hook, validation_monitor])
            monitors=[logging_hook])

        self.model = tsa_classifier

    def load_model(self, checkpoint=None):
        print(self.checkpoint_path, self.model_id)
        if checkpoint:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=self.checkpoint_path + "/" + self.model_id + "/model.ckpt-" + str(checkpoint))
        else:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=self.checkpoint_path + "/" + self.model_id)
        print ("LOADED MODEL AT STEP " + str(tsa_classifier.get_variable_value("global_step")) + " FROM " + self.checkpoint_path + "/" + self.model_id )
        self.model = tsa_classifier

    def bootstrap_model(self, path=None):
        if path:
            checkpoint_path = path
        else:
            checkpoint_path = self.checkpoint_path + "/" + self.model_id
        print ("USING CHECKPOINT PATH  " + checkpoint_path)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=checkpoint_path)
        print ("BOOTSTRAPPED AT STEP " + str(tsa_classifier.get_variable_value("global_step")))
        self.model = tsa_classifier 

    def predict(self):
        return self.model.predict(x=self.iteratorClass(self.ids, self.data_path, repeating=False, pool_size=2, scaler_ids=self.scaler_ids, file_format=".a3daps"))

In [ ]:
tensors_to_log =  {"probabilities": "prob_tensor",
                    "actual":"labels","reg_losses":"reg_losses"}

image_df = pd.read_csv('./stage1_flipped.csv')
image_df['zone'] = image_df['Id'].str.split("_Zone", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()
train_ids, test_ids, _, _ = train_test_split(ids, ids, test_size=.1)
train_ids.sort()
test_ids.sort()

train_labels = image_df[image_df['id'].isin(train_ids)]
train_labels.zone = train_labels.zone.astype("int")
train_labels = train_labels.sort_values(["id","zone"])
train_labels = train_labels["Probability"]
train_labels = np.reshape(train_labels, (-1,17))

test_labels = image_df[image_df['id'].isin(test_ids)]
test_labels.zone = test_labels.zone.astype("int")
test_labels = test_labels.sort_values(["id","zone"])
test_labels = test_labels["Probability"]
# test_labels = np.reshape(test_labels, (-1,17))



In [ ]:
eval_df = pd.read_csv('./stage1_sample_submission.csv')
eval_df['zone'] = eval_df['Id'].str.split("_", expand=True)[1].str.strip()
eval_df['id'] = eval_df['Id'].str.split("_", expand=True)[0].str.strip()

eval_ids = eval_df["id"].unique()
eval_ids.sort()

In [15]:
with tf.Session() as sess:
    model = ZoneModel(MODEL_ID, train_ids, "trimmed", "both", DATA_PATH, train_labels, CHECKPOINT_PATH, iteratorClass=tsa_utils.SampledImagesIterator, test_ids=test_ids, test_labels=test_labels, randomize=True, seed=10009)     
    sess.run( tf.initialize_variables( list( tf.get_variable(name) for name in sess.run( tf.report_uninitialized_variables( tf.all_variables( ) ) ) ) ) )
    model.train_model(tensors_to_log, reuse=False)

Instructions for updating:
Please use tf.global_variables instead.


From <ipython-input-15-92478c4fd9a2>:3: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


From /usr/local/lib/python3.5/site-packages/tensorflow/python/util/tf_should_use.py:170: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.


INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_environment': 'local', '_num_worker_replicas': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_task_type': None, '_master': '', '_model_dir': '/output/multi_class/multi_class3d10', '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdc0c1a8a90>, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_session_config': None, '_task_id': 0}
Using config: {'_is_chief': True, '_evaluation_master': '', '_save_checkpoints_secs': 600, '_environment': 'local', '_num_worker_replicas': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_task_type': None, '_master': '', '_model_

...

From <ipython-input-12-2b4f94093e30>:142: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with batch_size is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


From <ipython-input-12-2b4f94093e30>:142: calling BaseEstimator.fit (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


<function l2_regularizer.<locals>.l2 at 0x7fdc5d75d158>
True
256 330
Tensor("conv1/BiasAdd:0", shape=(10, 16, 330, 256, 30), dtype=float32)
Tensor("pool2/MaxPool3D:0", shape=(10, 16, 164, 127, 30), dtype=float32)
Tensor("conv4/Relu:0", shape=(10, 16, 164, 127, 30), dtype=float32)
Tensor("conv4/Relu:0", shape=(10, 16, 164, 127, 30), dtype=float32)
Tensor("pool3/MaxPool3D:0", shape=(10, 16, 23, 18, 30), dtype=float32)
Tensor("flat_pool:0", shape=(10, 198720), dtype=float32)
Tensor("dropout_2/dropout/mul:0", shape=(10, 198720), dtype=float32)
train
INFO:tensorflow:Create CheckpointSaverHook.
Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /output/multi_class/multi_class3d10/model.ckpt-2535
Restoring parameters from /output/multi_class/multi_class3d10/model.ckpt-2535
INFO:tensorflow:Saving checkpoints for 2536 into /output/multi_class/multi_class3d10/model.ckpt.
Saving checkpoints for 2536 into /output/multi_class/multi_class3d10/model.ckpt.
INFO:tensorflow:step = 253

KeyboardInterrupt: 

# Calculate F1

In [ ]:
def calculate_f1(model):
    predicted = np.array([x["probabilities"] for x in list(model.predict())])
    predicted[predicted < .4] = 0
    predicted[predicted > .4] = 1
    predicted = predicted.flatten()

    print(sklearn.metrics.f1_score(test_labels, predicted))
    print(sklearn.metrics.precision_score(test_labels, predicted))
    print(sklearn.metrics.recall_score(test_labels, predicted))


In [33]:

model = ZoneModel(MODEL_ID, test_ids, "both","both", DATA_PATH, test_labels, CHECKPOINT_PATH, scaler_ids=train_ids, iteratorClass=tsa_utils.InputImagesIterator)
model.load_model()
calculate_f1(model)



/output/multi_class multi_class3d3
INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_evaluation_master': '', '_tf_random_seed': None, '_num_ps_replicas': 0, '_environment': 'local', '_session_config': None, '_task_id': 0, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb6b1f38c50>, '_model_dir': '/output/multi_class/multi_class3d3', '_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_keep_checkpoint_max': 5}
Using config: {'_evaluation_master': '', '_tf_random_seed': None, '_num_ps_replicas': 0, '_environment': 'local', '_session_config': None, '_task_id': 0, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 1

From <ipython-input-28-d8000311d107>:160: calling BaseEstimator.predict (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


<function l2_regularizer.<locals>.l2 at 0x7fb6b1ebbae8>
False
256 330
Tensor("conv1/BiasAdd:0", shape=(1, 64, 330, 256, 5), dtype=float32)
Tensor("pool2/MaxPool3D:0", shape=(1, 32, 110, 85, 5), dtype=float32)
Tensor("conv4/Relu:0", shape=(1, 32, 110, 85, 5), dtype=float32)
Tensor("flat_pool:0", shape=(1, 1496000), dtype=float32)
Tensor("dropout_2/Identity:0", shape=(1, 1496000), dtype=float32)
infer
INFO:tensorflow:Restoring parameters from /output/multi_class/multi_class3d3/model.ckpt-1509
Restoring parameters from /output/multi_class/multi_class3d3/model.ckpt-1509
0.18115942029
0.342465753425
0.12315270936


# Validate

In [39]:
def validate_model(model):
    predicted = np.array([x["probabilities"] for x in list(model.predict())])
    predicted[predicted < .5] = 0
    predicted[predicted > .5] = 1
    predicted = predicted.flatten()
    flat_train_labels = train_labels.flatten()
    print(sklearn.metrics.f1_score(flat_train_labels, predicted))
    print(sklearn.metrics.precision_score(flat_train_labels, predicted))
    print(sklearn.metrics.recall_score(flat_train_labels, predicted))



In [42]:

model = ZoneModel(MODEL_ID, train_ids, "both","both", DATA_PATH, train_labels, CHECKPOINT_PATH, scaler_ids=test_ids, iteratorClass=tsa_utils.InputImagesIterator)
model.load_model()
validate_model(model)

/output/multi_class multi_class3d3
INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_evaluation_master': '', '_tf_random_seed': None, '_num_ps_replicas': 0, '_environment': 'local', '_session_config': None, '_task_id': 0, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb6b1c5ad30>, '_model_dir': '/output/multi_class/multi_class3d3', '_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_keep_checkpoint_max': 5}
Using config: {'_evaluation_master': '', '_tf_random_seed': None, '_num_ps_replicas': 0, '_environment': 'local', '_session_config': None, '_task_id': 0, '_num_worker_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 1

From <ipython-input-28-d8000311d107>:160: calling BaseEstimator.predict (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


<function l2_regularizer.<locals>.l2 at 0x7fb6b266e8c8>
False
256 330
Tensor("conv1/BiasAdd:0", shape=(1, 64, 330, 256, 5), dtype=float32)
Tensor("pool2/MaxPool3D:0", shape=(1, 32, 110, 85, 5), dtype=float32)
Tensor("conv4/Relu:0", shape=(1, 32, 110, 85, 5), dtype=float32)
Tensor("flat_pool:0", shape=(1, 1496000), dtype=float32)
Tensor("dropout_2/Identity:0", shape=(1, 1496000), dtype=float32)
infer
INFO:tensorflow:Restoring parameters from /output/multi_class/multi_class3d3/model.ckpt-1509
Restoring parameters from /output/multi_class/multi_class3d3/model.ckpt-1509
0.992453969212
0.999392097264
0.985611510791


# Build Kaggle Results

In [10]:
def build_results(model, filename):
    eval_df = pd.read_csv(DATA_PATH + 'stage1_sample_submission.csv')
    eval_df['zone'] = eval_df['Id'].str.split("_", expand=True)[1].str.strip()
    eval_df['id'] = eval_df['Id'].str.split("_", expand=True)[0].str.strip()

    eval_ids = eval_df["id"].unique()
    eval_ids.sort()

    csv_file = open(CHECKPOINT_PATH + "/" + filename + ".csv", "w+")
    csv_file.write("Id,Probability\n")

    print ("LOADED MODEL " + str(model))
    predicted = np.array([x["probabilities"] for x in list(model.predict())])
    for i, image in enumerate(predicted):
        for j, zone in enumerate(image):
            csv_file.write(str(eval_ids[i]) + "_Zone" + str(j+1) + "," + str(zone) + "\n")
    csv_file.close()



# Grid Search Parameters

In [ ]:
skip_list=[]
for i, iterator in enumerate([tsa_utils.ScaledImagesIterator,
                 tsa_utils.InputImagesIterator,
                 tsa_utils.ThresholdedInputImagesIterator,
                 tsa_utils.ThresholdedScaledImagesIterator                
                ]):
    for j, filter_count in enumerate([5,10,25,50]):
        FLAT_POOL_SIZE = int(48000 * filter_count / 10)
        for k, rv in enumerate([0.1,1.0,10.0]):
            for l, step_count in enumerate([300, 800, 1500, 2000]):
                STEPS=step_count
                try:
                    REGULARIZER_VALUE = rv
                    FILTER_COUNT = filter_count
                    with tf.Session() as sess:
                        MODEL_ID = str(i) + str(j) + str(k) + str(l) + "model4"
                        if MODEL_ID not in skip_list:
                            print(MODEL_ID)
                            model = ZoneModel(MODEL_ID, train_ids, "trimmed", "both", DATA_PATH, train_labels, CHECKPOINT_PATH, iteratorClass=iterator, scaler_ids=train_ids, test_ids=test_ids, test_labels=np.reshape(test_labels, (-1,17)))     
                            model.train_model(tensors_to_log, reuse=False)
                            model = ZoneModel(MODEL_ID, test_ids, "both","both", DATA_PATH, test_labels, CHECKPOINT_PATH, scaler_ids=train_ids, iteratorClass=iterator)
                            model.load_model()
                            print("F1 Calculated")
                            calculate_f1(model)
                            model = ZoneModel(MODEL_ID, train_ids, "trimmed", "both", DATA_PATH, train_labels, CHECKPOINT_PATH, iteratorClass=iterator, scaler_ids=train_ids, test_ids=test_ids, test_labels=np.reshape(test_labels, (-1,17)))     
                            model.load_model()
                            print("Validated:")
                            validate_model(model)
                            print("Saving results")
                            model = ZoneModel(MODEL_ID, eval_ids, "both","both", DATA_PATH, eval_ids, CHECKPOINT_PATH,scaler_ids=train_ids, iteratorClass=iterator)
                            model.load_model()
                            build_results(model, MODEL_ID)
                except Exception as e:
                    print(e)
                    print("ERRORED ON " + str(MODEL_ID))


0000model4


/usr/local/lib/python3.5/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


<function l2_regularizer.<locals>.l2 at 0x7f718653bae8>
256 330
True
Tensor("conv1/BiasAdd:0", shape=(10, 16, 330, 256, 5), dtype=float32)
Tensor("pool0/MaxPool3D:0", shape=(10, 16, 164, 127, 5), dtype=float32)
Tensor("conv2/Relu:0", shape=(10, 16, 165, 128, 5), dtype=float32)
Tensor("pool1/MaxPool3D:0", shape=(10, 16, 82, 63, 5), dtype=float32)
Tensor("conv3/BiasAdd:0", shape=(10, 16, 41, 32, 5), dtype=float32)
Tensor("pool2/MaxPool3D:0", shape=(10, 16, 20, 15, 5), dtype=float32)
Tensor("conv4/Relu:0", shape=(10, 16, 20, 15, 5), dtype=float32)
Tensor("flat_pool:0", shape=(10, 24000), dtype=float32)
Tensor("dropout_2/dropout/mul:0", shape=(10, 24000), dtype=float32)
train
/output/multi_class 0000model4
LOADED MODEL AT STEP 300 FROM /output/multi_class/0000model4
F1 Calculated
<function l2_regularizer.<locals>.l2 at 0x7f718653bae8>
256 330
False
Tensor("conv1/BiasAdd:0", shape=(1, 16, 330, 256, 5), dtype=float32)
Tensor("pool0/MaxPool3D:0", shape=(1, 16, 164, 127, 5), dtype=float32)
Ten

In [3]:
import numpy as np
array = np.array([0, 1 , 1 ,0])

In [4]:
array 

array([0, 4, 4, 0])